In [2]:
!pip uninstall tweepy

In [3]:
!pip install tweepy
!pip install nltk
!pip install transformers
!pip install emot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 12.8 MB/s 
     

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
import tweepy

In [6]:
from tweepy import Client

In [7]:

import pandas as pd
import re
import matplotlib.pyplot as plt
from wordcloud import WordCloud
%matplotlib inline
from transformers import pipeline, TFAutoModelForSequenceClassification , AutoTokenizer
from textblob import TextBlob
import re
import string # Inbuilt string library

# Natural Language Processing Toolkit
from nltk.corpus import stopwords, words # get stopwords from NLTK library & get all words in english language
from nltk.tokenize import word_tokenize # to create word tokens
# from nltk.stem import PorterStemmer (I played around with Stemmer and decided to use Lemmatizer instead)
from nltk.stem import WordNetLemmatizer # to reduce words to orginal form
from nltk import pos_tag # For Parts of Speech tagging

from emot.emo_unicode import UNICODE_EMOJI # For emojis
from emot.emo_unicode import EMOTICONS_EMO # For EMOTICONS

In [8]:
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [9]:
    num_of_pages = 3
    # Defining my NLTK stop words and my user-defined stop words
    stop_words = list(stopwords.words('english'))
    user_stop_words = ['2022', 'year', 'many', 'much', 'amp', 'next', 'cant', 'wont', 'hadnt',
                        'havent', 'hasnt', 'isnt', 'shouldnt', 'couldnt', 'wasnt', 'werent',
                        'mustnt', '’', '...', '..', '.', '.....', '....', 'been…', 'one', 'two',
                        'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'aht',
                        've', 'next']
    alphabets = list(string.ascii_lowercase)
    stop_words = stop_words + user_stop_words + alphabets
    word_list = words.words()  # all words in English language
    emojis = list(UNICODE_EMOJI.keys())  # full list of emojis

In [10]:
#getting recent tweets data
def getTweets():
    #asking for the search term and the desired number of tweets
    global keyword 
    keyword = 'VikramTrailer OR #ProofOfInspiration1 OR #Cricket OR #IPL2022 OR #IPL OR #RCB OR #CSK OR #MI OR #DC OR #SRH OR #KKR OR #RR OR #LSG OR #GT OR #PBKS OR @IPL OR @ChennaiIPL OR @imVkohli OR @ChennaiIPL OR @msdhoni OR @BCCI OR @ImRo45 OR Cricket OR IPL2022 OR IPL OR RCB OR CSK OR MI OR DC OR SRH OR KKR OR RR OR LSG OR GT OR PBKS OR IPL OR ChennaiIPL OR imVkohli OR ChennaiIPL OR msdhoni OR BCCI OR ImRo45'
    query = f'{keyword} -is:retweet lang:en'
    num_of_tweets = 100
    # your start and end time for fetching tweets
    start_time = '2022-05-18T00:00:00Z'
    end_time = '2022-05-24T00:00:00Z'
    #connecting to the twitter API using clent and the bearer_token credentials from config.py
    client = tweepy.Client("AAAAAAAAAAAAAAAAAAAAAGyWZwEAAAAAAYa51qdvaXMLT5VYTa1vMGahYb8%3DOFW2DenxaoxylnltQD8MhvXTtA838sWho7Fjv62SD2m6rQYuWU",wait_on_rate_limit=True)

    #using tweepy paginator to get over 100 last tweets from twitter api
    tweets = []
    users = []
    for tweet in tweepy.Paginator(client.search_recent_tweets,
                                    query = query,    
                                    start_time=start_time,
                                    end_time=end_time,
                                    tweet_fields = ['author_id', 'id','created_at', 'public_metrics', 'text', 'source','geo'],
                                    user_fields = ['name', 'username', 'location', 'verified', 'description'],
                                    place_fields=['geo','name'],
                                    expansions=['author_id','geo.place_id'],
                                    max_results = num_of_tweets, limit=num_of_pages):
        tweets.append(tweet)
        #print(dir(tweet))
        #users.append(tweet.includes['users'])
    return tweets


#function to clean the tweets and load them into a DataFrame
def tweetsETL(tweets):
    
    result = []
    
    #regex function to clean the tweet text from haashtags, mentions and links
    def cleanTweets(text):
        clean_text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())
        return clean_text
    def getHashTags(text):
        regex = "#(\w+)"
        return re.findall(regex, str(text))
    def getMentions(text):
        regex = "@(\w+)"
        return re.findall(regex, str(text))
    # Function to remove punctuations, links, emojis, and stop words
    def preprocessTweets(tweet):
        tweet = tweet.lower()  #has to be in place
        # Remove urls
        tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
        # Remove user @ references and '#' from tweet
        tweet = re.sub(r'\@\w+|\#|\d+', '', tweet)
        # Remove stopwords
        tweet_tokens = word_tokenize(tweet)  # convert string to tokens
        filtered_words = [w for w in tweet_tokens if w not in stop_words]
        filtered_words = [w for w in filtered_words if w not in emojis]
        filtered_words = [w for w in filtered_words if w in word_list]
        # Remove punctuations
        unpunctuated_words = [char for char in filtered_words if char not in string.punctuation]
        unpunctuated_words = ' '.join(unpunctuated_words)
        return "".join(unpunctuated_words)  # join words with a space in between them   
    def preprocessTweetsSentiments(tweet):
        tweet_tokens = word_tokenize(tweet)
        lemmatizer = WordNetLemmatizer() # instatiate an object WordNetLemmatizer Class
        lemma_words = [lemmatizer.lemmatize(w) for w in tweet_tokens]
        return " ".join(lemma_words)      
    def getPolarity(text):
        return TextBlob(text).sentiment.polarity    
    def getSubjectivity(text):
        return TextBlob(text).sentiment.subjectivity  
    
    def getAnalysis(score):
        if score < 0:
            return 'Negative'
        elif score == 0:
            return 'Neutral'
        else:
            return 'Positive'
    #function to unpack the tweets list into a dataframe
    for i in range(0, num_of_pages) :        
        for tweet, user in zip(tweets[i].data, tweets[i].includes['users']):
                p_tweet=preprocessTweets(tweet.text)
                s_tweet=preprocessTweetsSentiments(p_tweet)
                result.append({'tweet_id': tweet.id,
                               'author_id': tweet.author_id,                               
                               'tweet_by':user.username,
                               'tweet_by_verified':user.verified,
                               'user_location':user.location,
                               'text': tweet.text,
                               'clean_tweet' : p_tweet,
                              #  'adjectives' : getAdjectives(p_tweet),
                               'sentiment_text' : s_tweet,
                               'created_at': tweet.created_at,
                               'source':tweet.source,
                               'retweets': tweet.public_metrics['retweet_count'],
                               'replies': tweet.public_metrics['reply_count'],
                               'likes': tweet.public_metrics['like_count'],
                               'quote_count': tweet.public_metrics['quote_count'],
                               'hastags':getHashTags(tweet.text),
                               'mentions':getMentions(tweet.text),
                               'subjectivity':getSubjectivity(s_tweet),
                               'polarity':getPolarity(s_tweet),
                               'sentiment':getAnalysis(getPolarity(s_tweet))
                          })

    df = pd.DataFrame(result)     
    return df
    

In [11]:
tweets=getTweets()

BadRequest: ignored

In [ ]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

In [ ]:
print(len(tweets))
df=tweetsETL(tweets)
#df.duplicated(subset='tweet_id').sum()
#df=tweets_df.drop_duplicates(subset=['tweet_id']) # drop duplicate values
#df.isna().any() # Ch 
df

In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Graph Analysis/Tweets_Processed_Temp3.csv',encoding='utf-8-sig', index=False)

In [ ]:
df.count()

In [ ]:
df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')